In [1]:
import pandas as pd
import numpy as np

In [2]:
exclude = ['Posted On', 'Area Locality', 'Floor']

In [3]:
data = pd.read_csv('../data/raw.csv')

In [4]:
data

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2,10000,1100,Ground out of 2,Super Area,Bandel,Kolkata,Unfurnished,Bachelors/Family,2,Contact Owner
1,2022-05-13,2,20000,800,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
2,2022-05-16,2,17000,1000,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1,Contact Owner
3,2022-07-04,2,10000,800,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1,Contact Owner
4,2022-05-09,2,7500,850,1 out of 2,Carpet Area,South Dum Dum,Kolkata,Unfurnished,Bachelors,1,Contact Owner
...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2022-05-18,2,15000,1000,3 out of 5,Carpet Area,Bandam Kommu,Hyderabad,Semi-Furnished,Bachelors/Family,2,Contact Owner
4742,2022-05-15,3,29000,2000,1 out of 4,Super Area,"Manikonda, Hyderabad",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Owner
4743,2022-07-10,3,35000,1750,3 out of 5,Carpet Area,"Himayath Nagar, NH 7",Hyderabad,Semi-Furnished,Bachelors/Family,3,Contact Agent
4744,2022-07-06,3,45000,1500,23 out of 34,Carpet Area,Gachibowli,Hyderabad,Semi-Furnished,Family,2,Contact Agent


In [5]:
import mlflow

# mlflow.set_tracking_uri("http://127.0.0.1:5000")

mlflow.set_tracking_uri("http://ec2-184-73-140-65.compute-1.amazonaws.com:5000")

In [6]:
from pycaret.regression import *
sv = setup( data = data, ignore_features = exclude,
           target = 'Rent', session_id = 1234,
           experiment_name = 'experiments_rent_model',
           log_experiment = True,
           )

,Description,Value
0,Session id,1234
1,Target,Rent
2,Target type,Regression
3,Original data shape,"(4746, 12)"
4,Transformed data shape,"(4746, 22)"
5,Transformed train set shape,"(3322, 22)"
6,Transformed test set shape,"(1424, 22)"
7,Ignore features,3
8,Numeric features,3
9,Categorical features,5


In [7]:
# best = compare_models()

In [8]:
best = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11670.1429,752962571.5195,27440.1635,0.7389,0.4776,0.4850
1,13672.1154,1436274865.7232,37898.2172,0.6854,0.4219,0.3488
2,12962.0633,1973093346.8632,44419.5154,0.6949,0.4109,0.3569
3,12190.3987,1125621431.6930,33550.2821,0.5766,0.4561,0.4345
4,14011.4918,2745410304.9051,52396.6631,-0.5926,0.4716,0.4480
5,13198.6646,1995584310.1411,44671.9634,0.5143,0.4050,0.3540
6,13587.9068,2347935823.8514,48455.5035,0.2182,0.4223,0.3924
7,15678.0590,4563207941.0173,67551.5206,-0.9821,0.4465,0.4091
8,21451.5667,36272084003.4115,190452.3143,0.0989,0.5111,0.4651


In [9]:
tuned_model = tune_model(best, n_iter=20, optimize='MAE') 

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,11956.8935,1110224277.0560,33320.0282,0.6150,0.4691,0.4157
1,12774.1036,1275635866.0920,35716.0449,0.7206,0.4099,0.3100
2,12992.8147,3280567246.1403,57276.2363,0.4927,0.3805,0.2880
3,10107.4077,609599399.6089,24690.0668,0.7707,0.3924,0.3285
4,9899.4043,519601586.0181,22794.7710,0.6986,0.4031,0.3085
5,11588.7836,1481289295.1218,38487.5213,0.6395,0.3720,0.2938
6,12003.6574,767054016.3694,27695.7400,0.7446,0.4180,0.3560
7,10173.5843,619397382.3969,24887.6954,0.7309,0.3898,0.2892
8,21597.5652,36545722191.4984,191169.3547,0.0921,0.4383,0.3194


Fitting 10 folds for each of 20 candidates, totalling 200 fits


2026/01/19 22:29:51 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2026/01/19 22:29:51 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Regressor at: http://ec2-184-73-140-65.compute-1.amazonaws.com:5000/#/experiments/334592644149538691/runs/1444e13bff234a0d95043cb17adf0ea1.
2026/01/19 22:29:51 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-184-73-140-65.compute-1.amazonaws.com:5000/#/experiments/334592644149538691.


In [9]:
tuned_model

RandomForestRegressor(bootstrap=False, criterion='absolute_error', max_depth=5,
                      max_features='sqrt', min_impurity_decrease=0.2,
                      min_samples_leaf=6, min_samples_split=9, n_estimators=110,
                      n_jobs=-1, random_state=123)

In [10]:
final = finalize_model(tuned_model)

2026/01/19 21:13:42 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2026/01/19 21:13:42 INFO mlflow.tracking._tracking_service.client: 🏃 View run Random Forest Regressor at: http://127.0.0.1:5000/#/experiments/847632105172078396/runs/9013ba58f72d436c9467d002268c36c8.
2026/01/19 21:13:42 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/847632105172078396.


In [12]:
final

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=['BHK', 'Size', 'Bathroom'],
                                    transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['Area Type', 'City',
                                             'Furnishing Status',
                                             'Tenant Preferred',
                                             'Point of Contact'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('ordinal_encoding',
                 Tra...
                                                              handle_missing='return_nan',
                                                              use_cat_names=True))),
                ('clean_column_names',
                 TransformerWrapper(transformer=CleanColumnNames())),
                ('actual_estimator',
                 RandomForestRegressor(bootstrap=False,
                                       criterion='absolute_error', max_depth=5,
                                       max_features='sqrt',
                                       min_impurity_decrease=0.2,
                                       min_samples_leaf=6, min_samples_split=9,
                                       n_estimators=110, n_jobs=-1,
                                       random_state=123))])

In [13]:
save_model(final, 'model_rf')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['BHK', 'Size', 'Bathroom'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=['Area Type', 'City',
                                              'Furnishing Status',
                                              'Tenant Preferred',
                                              'Point of Contact'],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('ordinal_encoding',
                  Tra...
                                                               handle_missing='return_nan',
                                                               use_cat_names=True))),
                 ('clean_column_names',
                  TransformerWrapper(transformer=CleanColumnNames())),
                 ('actual_e